In [1]:
import pandas as pd
import numpy as np
import Costs
import Economics
import copy

In [2]:
# Load in Type Curve
type_well = pd.read_csv("./type_curve.csv")

# Load in Well Parameters
well_params = pd.read_csv("./econ_params.csv")

In [4]:
def LoadWell(type_well, well_params):
    newWell = pd.merge(type_well, well_params, left_on = "DATE", right_on = "DATE", how = "outer").fillna(method='ffill')
    newWell['DATE']=pd.to_datetime(newWell['DATE'])
    # Add 12 Rows for Capital
    for i in range(12):
        a = newWell.loc[1, 'DATE'] -  pd.offsets.MonthBegin()
        newWell = pd.concat([pd.DataFrame([{'DATE':a, 'GAS': 0, 'WATER': 0}]), newWell], ignore_index=True).fillna(method='bfill')
    
    return newWell

In [5]:
# Populate Capital
def PopulateCapital(well, LATERAL_LENGTH=9000, CAPITAL_FACTOR=1):
    costs = Costs.Costs()
    well['CAPITAL'] = 0
    well.loc[0, 'CAPITAL'] = costs.utica_total_drill(LATERAL_LENGTH) * CAPITAL_FACTOR
    well.loc[8, 'CAPITAL'] = costs.utica_total_completion(LATERAL_LENGTH) * CAPITAL_FACTOR
    well.loc[10, 'CAPITAL'] = costs.prod_equipment(water_injection = 1) * CAPITAL_FACTOR
    well.loc[well.shape[0]-1, 'CAPITAL'] = costs.abandonment() * CAPITAL_FACTOR
    well['GAS'] = well['GAS'] * LATERAL_LENGTH/1000 * 365.25/12
    well['WATER'] = well['WATER'] * LATERAL_LENGTH/1000 * 365.25/12
    return well

In [6]:
def CalculateOpex(well):
    well['OPEX_TOTAL'] = 0
    well['OPEX_TOTAL'] = well['WI'] * (well['FIXED_OPEX'] + (well['VAR_OPEX'] + well['TRANSPORT_GAS']) * well['GAS'] + \
                            well['WATER_OPEX'] * well['WATER'])
    return well

In [7]:
def CalculateRevenue(well):
    well['REVENUE'] = 0
    well['REVENUE'] = well['NRI'] * ( well['GAS_PRICE'] + well['GAS_DEDUCT'] ) * well ['BTU'] * \
                        well['SHRINK'] * well['GAS']
    return well

In [8]:
def CalculateCashflow(well):
    #Operating cashflow is not negative.  Well is not operated if negative cashflow
    well['OPERATING_CASHFLOW'] = well['REVENUE'] - well['OPEX_TOTAL']
    well.loc[well['OPERATING_CASHFLOW'] < 0, 'OPERATING_CASHFLOW'] = 0
    well['TOTAL_CASHFLOW'] = well['OPERATING_CASHFLOW'] - well['CAPITAL']
    
    return well

# Generate Grid of Well Economics

In [9]:
economics = Economics.Economics()

In [10]:
# Iterate Through Type Curve Factor
for typeCurveFactor in np.linspace(0.8,1.2,9):
    for gasPrice in np.linspace(1.25,2.75,11):
        # Create a new Well
        well = LoadWell(type_well, well_params)
        
        # Update the Grid Search Parameters
        well['GAS_PRICE'] = gasPrice
        well['GAS'] = well['GAS'] * typeCurveFactor
        well['WATER'] = well['WATER'] * typeCurveFactor
        
        # Calculate the Economics
        well = PopulateCapital(well, LATERAL_LENGTH=9000, CAPITAL_FACTOR=1)
        well = CalculateOpex(well)
        well = CalculateRevenue(well)
        well = CalculateCashflow(well)
        well['ID'] = f"{typeCurveFactor:.2f},{gasPrice:.2f}"
        
        # Calculate Economic Indicators and add to report
        economics.generate_well_economics(well)

C:\Users\brent\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [11]:
economics.single_well_df

ID           NPV       IRR      MOIC       F_D           EUR  \
0   0.80,1.25 -5.527279e+06  0.100110 -0.402021  0.730525  1.211694e+07   
1   0.80,1.40 -4.612767e+06  0.100148 -0.196174  0.730525  1.211694e+07   
2   0.80,1.55 -3.698129e+06  0.000258  0.011170  0.730525  1.211694e+07   
3   0.80,1.70 -2.783491e+06  0.025190  0.218514  0.730525  1.211694e+07   
4   0.80,1.85 -1.868852e+06  0.049395  0.425858  0.730525  1.211694e+07   
..        ...           ...       ...       ...       ...           ...   
94  1.20,2.15  4.318951e+06  0.244414  1.828603  0.487017  1.817541e+07   
95  1.20,2.30  5.690908e+06  0.297092  2.139619  0.487017  1.817541e+07   
96  1.20,2.45  7.062866e+06  0.351875  2.450635  0.487017  1.817541e+07   
97  1.20,2.60  8.434824e+06  0.408743  2.761651  0.487017  1.817541e+07   
98  1.20,2.75  9.806781e+06  0.468902  3.072667  0.487017  1.817541e+07   

         CAPITAL  CUM_CASHFLOW    OPEX_TOTAL  
0   8.851728e+06 -3.558579e+06  8.203358e+06  
1   8.851728e+06 -1.736477e+06  8.203358e+06  
2   8.851728e+06  9.887577e+04  8.203358e+06  
3   8.851728e+06  1.934229e+06  8.203358e+06  
4   8.851728e+06  3.769582e+06  8.203358e+06  
..           ...           ...           ...  
94  8.851728e+06  1.618630e+07  1.169304e+07  
95  8.851728e+06  1.893933e+07  1.169304e+07  
96  8.851728e+06  2.169235e+07  1.169304e+07  
97  8.851728e+06  2.444538e+07  1.169304e+07  
98  8.851728e+06  2.719841e+07  1.169304e+07  

[99 rows x 9 columns]

# Create Results Table

In [24]:
table_df = copy.copy(economics.single_well_df)

In [27]:
table_df['Type Curve Factor'], table_df['Gas Price'] = table_df['ID'].str.split(',', 1).str

# NPV 10

In [29]:
 pd.pivot_table(table_df, values='NPV', index=['Type Curve Factor'], columns=['Gas Price'], aggfunc=np.sum)

Gas Price                  1.25          1.40          1.55          1.70  \
Type Curve Factor                                                           
0.80              -5.527279e+06 -4.612767e+06 -3.698129e+06 -2.783491e+06   
0.85              -5.325509e+06 -4.353776e+06 -3.381973e+06 -2.410170e+06   
0.90              -5.123720e+06 -4.094785e+06 -3.065817e+06 -2.036848e+06   
0.95              -4.921915e+06 -3.835794e+06 -2.749660e+06 -1.663527e+06   
1.00              -4.720099e+06 -3.576802e+06 -2.433504e+06 -1.290206e+06   
1.05              -4.518274e+06 -3.317811e+06 -2.117348e+06 -9.168852e+05   
1.10              -4.316447e+06 -3.058820e+06 -1.801192e+06 -5.435641e+05   
1.15              -4.114621e+06 -2.799828e+06 -1.485036e+06 -1.702430e+05   
1.20              -3.912795e+06 -2.540837e+06 -1.168879e+06  2.030781e+05   

Gas Price                  1.85          2.00          2.15          2.30  \
Type Curve Factor                                                           
0.80              -1.868852e+06 -9.542139e+05 -3.957553e+04  8.750629e+05   
0.85              -1.438366e+06 -4.665630e+05  5.052403e+05  1.477044e+06   
0.90              -1.007880e+06  2.108787e+04  1.050056e+06  2.079024e+06   
0.95              -5.773943e+05  5.087388e+05  1.594872e+06  2.681005e+06   
1.00              -1.469083e+05  9.963897e+05  2.139688e+06  3.282986e+06   
1.05               2.835777e+05  1.484041e+06  2.684503e+06  3.884966e+06   
1.10               7.140637e+05  1.971691e+06  3.229319e+06  4.486947e+06   
1.15               1.144550e+06  2.459342e+06  3.774135e+06  5.088928e+06   
1.20               1.575036e+06  2.946993e+06  4.318951e+06  5.690908e+06   

Gas Price                  2.45          2.60          2.75  
Type Curve Factor                                            
0.80               1.789701e+06  2.704340e+06  3.618978e+06  
0.85               2.448847e+06  3.420650e+06  4.392453e+06  
0.90               3.107992e+06  4.136961e+06  5.165929e+06  
0.95               3.767138e+06  4.853271e+06  5.939404e+06  
1.00               4.426284e+06  5.569582e+06  6.712880e+06  
1.05               5.085429e+06  6.285892e+06  7.486355e+06  
1.10               5.744575e+06  7.002203e+06  8.259830e+06  
1.15               6.403720e+06  7.718513e+06  9.033306e+06  
1.20               7.062866e+06  8.434824e+06  9.806781e+06

# IRR

In [30]:
 pd.pivot_table(table_df, values='IRR', index=['Type Curve Factor'], columns=['Gas Price'], aggfunc=np.sum)

Gas Price              1.25      1.40      1.55      1.70      1.85      2.00  \
Type Curve Factor                                                               
0.80               0.100110  0.100148  0.000258  0.025190  0.049395  0.073573   
0.85               0.100110  0.100691  0.011739  0.033098  0.059795  0.086531   
0.90               0.100110  0.107688  0.017739  0.045893  0.072256  0.100621   
0.95               0.100111 -0.001494  0.025946  0.054159  0.083456  0.115181   
1.00               0.100121  0.003567  0.032396  0.063663  0.095578  0.129779   
1.05               0.100219  0.012652  0.044384  0.074504  0.108082  0.144152   
1.10               0.100946  0.017902  0.050915  0.084388  0.121340  0.161728   
1.15               0.106240  0.024832  0.058599  0.094860  0.134179  0.177830   
1.20               0.317616  0.030913  0.068478  0.105846  0.146795  0.194094   

Gas Price              2.15      2.30      2.45      2.60      2.75  
Type Curve Factor                                                    
0.80               0.098825  0.126160  0.152971  0.185987  0.219000  
0.85               0.115076  0.143948  0.177482  0.211914  0.247116  
0.90               0.131375  0.165254  0.200855  0.237750  0.275871  
0.95               0.147369  0.185208  0.224329  0.264181  0.306843  
1.00               0.167251  0.207027  0.248361  0.292361  0.337686  
1.05               0.185325  0.228587  0.272853  0.320569  0.369225  
1.10               0.205129  0.250598  0.299191  0.349406  0.401387  
1.15               0.224581  0.272984  0.325271  0.378816  0.435091  
1.20               0.244414  0.297092  0.351875  0.408743  0.468902